In [ ]:
import numpy as np
from corpus import Synchronizer

def calculate_wps(origin_path: str, cover_path: str, song_dir: str, lambda_val: float = 0.1) -> float:
    try:
        s = Synchronizer()
        wp = s.get_wp(origin_path, cover_path, song_dir)
        
        t_cover = s.t1
        t_orig = s.t2
        
        if t_cover is None or t_orig is None:
            raise ValueError("時間戳序列未能成功在 Synchronizer 物件中生成。")

        wp_int = wp.astype(int)

        indices_cover = wp_int[0]
        indices_orig = wp_int[1]

        time_diff_sequence = t_orig[indices_orig] - t_cover[indices_cover]

        sigma_d = np.std(time_diff_sequence)
        print(f"wp-std: {sigma_d}")
        wps_score = np.exp(-lambda_val * sigma_d)

        return wps_score

    except Exception as e:
        print(f"計算 WPS 分數時發生錯誤：{e}")
        return 0.0

if __name__ == '__main__':
    target_dir = "CPOP4"
    song_dir = f"./dataset/eval/{target_dir}"
    original_audio = f"{song_dir}/origin.wav"
    versions = ["picogen", "amtapc", "music2midi", "human"]

    for v in versions:
        cover_audio = f"{song_dir}/{v}.wav"
        wps_score = calculate_wps(original_audio, cover_audio, song_dir, 0.25)

        if wps_score > 0:
            print(f"WPS of {v}: {wps_score:.4f}")

In [ ]:
def calculate_nwpd(origin_path: str, cover_path: str, song_dir: str, lambda_nwpd: float = 1.0) -> float:
    try:
        s = Synchronizer()
        wp = s.get_wp(origin_path, cover_path, song_dir)
        
        t_cover = s.t1
        t_orig = s.t2
        
        if t_cover is None or t_orig is None:
            raise ValueError("時間戳序列未能成功在 Synchronizer 物件中生成。")

        wp_int = wp.astype(int)

        path_t_cover = t_cover[wp_int[0]]
        path_t_orig = t_orig[wp_int[1]]

        coeffs = np.polyfit(path_t_cover, path_t_orig, 1)
        a, b = coeffs[0], coeffs[1]

        t_orig_predicted = a * path_t_cover + b
        
        deviation = path_t_orig - t_orig_predicted

        sigma_dev = np.std(deviation)

        nwpd_score = np.exp(-lambda_nwpd * sigma_dev)

        return nwpd_score

    except Exception as e:
        print(f"計算 NWPD 分數時發生錯誤：{e}")
        return 0.0

if __name__ == '__main__':
    target_dir = "JPOP1"
    song_dir = f"./dataset/eval/{target_dir}"
    original_audio = f"{song_dir}/origin.wav"
    versions = ["picogen", "amtapc", "music2midi", "human"]

    for v in versions:
        cover_audio = f"{song_dir}/{v}.wav"
        nwpd_score = calculate_nwpd(original_audio, cover_audio, song_dir, 0.25)

        if nwpd_score > 0:
            print(f"NWPD of {v}: {nwpd_score:.4f}")

In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from corpus import Synchronizer

def calculate_nwpd(origin_path: str, cover_path: str, song_dir: str, lambda_nwpd: float = 1.0) -> float:
    """
    計算正規化路徑偏差 (Normalized Warp Path Deviation, NWPD) 分數。
    此版本已根據 wp 的實際 shape (2, L) 進行修正。
    """
    try:
        s = Synchronizer()
        wp = s.get_wp(origin_path, cover_path, song_dir)
        
        t_cover = s.t1
        t_orig = s.t2
        
        if t_cover is None or t_orig is None:
            raise ValueError("時間戳序列未能成功在 Synchronizer 物件中生成。")

        wp_int = wp.astype(int)
        path_t_cover = t_cover[wp_int[0]]
        path_t_orig = t_orig[wp_int[1]]

        coeffs = np.polyfit(path_t_cover, path_t_orig, 1)
        a, b = coeffs[0], coeffs[1]

        t_orig_predicted = a * path_t_cover + b
        deviation = path_t_orig - t_orig_predicted
        sigma_dev = np.std(deviation)
        nwpd_score = np.exp(-lambda_nwpd * sigma_dev)
        return nwpd_score
    except Exception as e:
        print(f"計算 NWPD 分數時發生錯誤：{e}")
        return 0.0


def analyze_and_visualize_scores():
    """
    主函式：計算所有歌曲各版本的 NWPD 分數，計算平均值，並進行視覺化。
    """
    # --- 1. 設定 ---
    base_dir = os.path.join(".", "dataset", "eval")
    metadata_path = os.path.join(base_dir, "metadata.json")
    origin_filename = "origin.wav"
    versions = ["human", "picogen", "amtapc", "music2midi"]
    lambda_val = 0.5
    
    # 用於儲存所有分數的字典
    scores_by_version = {v: [] for v in versions}

    # --- 2. 讀取 metadata 並計算分數 ---
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
        # print(f"✅ 成功讀取 metadata.json，共找到 {len(metadata)} 首歌曲。")
    except FileNotFoundError:
        print(f"❌ 錯誤：找不到 metadata.json 檔案。")
        return

    for i, song_data in enumerate(metadata):
        dir_name = song_data.get("dir_name")
        if not dir_name:
            continue

        song_dir = os.path.join(base_dir, dir_name)
        # print(f"\n🎵 正在處理歌曲: {song_dir} ({i+1}/{len(metadata)})")

        origin_path = os.path.join(song_dir, origin_filename)
        if not os.path.exists(origin_path):
            print(f"  ↪️ 已跳過 (找不到 origin.wav)")
            continue

        for v in versions:
            cover_path = os.path.join(song_dir, f"{v}.wav")
            if not os.path.exists(cover_path):
                print(f"  ↪️ 已跳過版本 '{v}' (找不到 {v}.wav) {song_dir}")
                continue
            
            # 計算分數
            score = calculate_nwpd(origin_path, cover_path, song_dir, lambda_nwpd=lambda_val)
            if score > 0:
                # print(f"  📊 版本 '{v}' 的 NWPD 分數: {score:.4f}")
                scores_by_version[v].append(score)

    # --- 3. 計算並打印平均分數 ---
    print("\n\n--- 平均分數統計 ---")
    average_scores = {}
    for version, scores in scores_by_version.items():
        if scores:
            avg_score = np.mean(scores)
            average_scores[version] = avg_score
            print(f"版本 {version:<12}: 平均 NWPD 分數 = {avg_score:.4f} (基於 {len(scores)} 個樣本)")
        else:
            print(f"版本 {version:<12}: 無有效分數可供計算。")
    
    # --- 4. 數據視覺化 ---
    print("\n🎨 正在生成分數分佈圖...")
    
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 7))

    # 為每個版本繪製一條 KDE 曲線
    for version, scores in scores_by_version.items():
        if scores:
            sns.kdeplot(scores, label=version, fill=True, alpha=0.5, ax=ax, lw=2.5)

    ax.set_title('NWPD Score Distribution by Version', fontsize=16, pad=20)
    ax.set_xlabel('NWPD Score', fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.set_xlim(0, 1.05)
    ax.legend(title='Version', fontsize=10)
    
    # 儲存圖表
    output_image_path = "nwpd_score_distribution.png"
    plt.savefig(output_image_path, dpi=300, bbox_inches='tight')
    
    print(f"✅ 分數分佈圖已成功儲存至: {output_image_path}")


if __name__ == '__main__':
    analyze_and_visualize_scores()